In [9]:
import os


imdb_dir = './resources/aclImdb'

In [2]:
# Process the labels of the raw IMDB data

train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if not fname.endswith('.txt'):
            continue
        with open(os.path.join(dir_name, fname)) as f:
            texts.append(f.read())
        labels.append(0 if label_type == 'neg' else 1)

In [11]:
# Tokenizing the text of the raw IMDB data

# tokenizer = get_tokenizer('basic_english')  # TODO
# sequences = [tokenizer(text) for text in texts]  # list of texts -> list of sequences
